# Warsztat 9 – Pandas - wprowadzenie<a id=top></a>

<font size=2>Przed pracą z notatnikiem polecam wykonać kod w ostatniej komórce (zawiera html i css), dzięki czemu całość będzie bardziej estetyczna :)</font>

<a href='#Warsztat-91---Pandas---wprowadzenie'>Warsztat 9</a>
<ul>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
<li><a href='#'><span></span></a></li>
</ul>

<img src="http://pandas.pydata.org/_static/pandas_logo.png" width=40% height=40%>

## Pakiet Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

### Wczytanie danych

Najczęściej będziemy posługiwać się plikami **.csv** oraz **.xlsx**, z których będziemy wczytywać uprzednio wydobyte dane oraz do których będziemy zapisywać wyniki naszej pracy.  
Składnia dla obu typów plików jest analogiczna i sprowadza się do użycia funkcji <b>read_csv( )</b> oraz <b>read_excel( )</b>.

In [ ]:
dane = pd.read_csv('pandas_data.csv')

Najwygodniejszym sposobem podglądu danych będzie użycie metod <b>.head( )</b> i <b>.tail( )</b>, które pokażą nam odpowiednio początek i koniec naszej ramki.

In [ ]:
dane.head()

In [ ]:
dane.tail(10)

Dane wczytały się bez problemu, jednak mamy jedną niepotrzebną kolumnę na samym początku ramki. Zostały w niej zapisane indeksy poszczególnych wierszy arkusza danych. Możemy się go łatwo pozbyć, używając komendy <b>index_col=</b> i wskazując numer kolumny, w której zapisane są indeksy wierszy.

In [ ]:
dane = pd.read_csv('pandas_data.csv', index_col=0)

In [ ]:
dane.head()

Gdyby jednak niechciana kolumna zawierała zupełnie nieprzydatne dane, możemy posłużyć się slicingiem kolumn po indeksach, wykorzystując metodę <b>.ix</b>, aby móc operować na indeksach a nie nazwach kolumn.  
Do wybierania elementów ramki będziemy używać nawiasów kwadratowych (podobnie jak w listach czy tablicach numpy).

In [ ]:
dane.ix[:,1:].head()

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

### Wybieranie danych 

Ramka zawiera bardzo wiele danych zebranych w trakcie eksperymentu. Zacznijmy od wybrania tylko tych, które mogą okazać się przydatne w analizie wyników. Najpierw wylistujemy sobie wszystkie nazwy kolumn, a następnie za ich pomocą stworzymy nową ramkę z mniejszą ilością danych.

In [ ]:
dane.columns.values

In [ ]:
dane_small = dane[['ID','task','duration_type','accuracy','accuracy_RT','PAS','PAS_RT']]

In [ ]:
dane_small.head()

Zanim zaczniemy pracować z całym zbiorem, wybierzmy dane dla jednej osoby badanej, na której będziemy testować kolejne kroki analizy danych.  
Sprawdźmy najpierw, jakie osoby badane mamy dostępne.

In [ ]:
osoby=dane_small.ID.unique()

In [ ]:
d19 = dane_small[dane_small.ID == 'dc1_19']

In [ ]:
d19.head()

Spójrzmy najpierw na poprawność osoby nr 19. W tym celu wybierzmy z danych jedynie interesujące nas zmienne niezależne (typ zadania i czas prezentacji bodźca) oraz zmienną zależną (poprawność).  
Następnym krokiem powinno być wyliczenie średniej poprawności dla każdej możliwej kombinacji zmiennych zależnych. Możemy to uzyskać grupując dane właśnie według zadania oraz czasu prezentacji bodźca. Posłuży nam do tego metoda <b>groupby( )</b>, której argumentem będzie kolumna (albo lista kolumn), wedle których nasze dane mają być pogrupowane.

In [ ]:
d19[['task','duration_type','accuracy']].groupby(['task','duration_type'])

Efektem działania tej metody jest specjalny obiekt, który przechowuje pogrupowane dane, jednak Python nie wie jeszcze, co ma z nimi zrobić. Żeby otrzymać sensowny wynik, musimy zaaplikować do danych interesującą nas funkcję. Pandas posiada sporo możliwych do użycia opcji, my wybieramy średnią (<b>.mean( )</b>).

In [ ]:
acc19 = d19[['task','duration_type','accuracy']].groupby(['task','duration_type']).mean()
acc19

Następnym krokiem będzie prosta wizualizacja wyników na wykresie liniowym. Pandas posiada interfejs do szybkiego tworzenia wykresów za pomocą biblioteki **matplotlib**, którą zaimportowaliśmy na samym początku tego warsztatu. Składnia jest bardzo prosta - na interesjącej nas ramce używamy metody <b>.plot()</b>.

In [ ]:
acc19.plot()

Otrzymujemy wykres zgodny z danymi, jednak taka wizualizacja nie może być satysfakcjonująca. Uczy nas jednak ważnej rzeczy, Pandas traktuje pojedynczą kolumnę jako jedną serię danych. My potrzebujemy osobnych linii dla każdego warunku zadania.  
Zadanie zakodowane jest w indeksie danych. Żeby transponować je do kolumny, użyjemy metody <b>.unstack( )</b>, której argumentem będzie nazwa zmiennej indeksującej, którą chcemy przenieść.

In [ ]:
acc19.stack().unstack('task')

In [ ]:
acc19.unstack('task').plot()

Efekt jest zgodny z oczekiwanym.  
Spróbujmy zwiększyć informacyjność wykresu wzbogacając go o słupki błędów. Jako miarę wybierzemy błąd standardowy, który obliczamy zgodnie ze wzorem: $$SE=\frac{STD}{\sqrt{N}}$$ gdzie:  
SE = błąd standardowy  
STD = odchylenie standardowe  
N = ilość obserwacji

Odchylenie standarodwe możemy obliczyć identycznym sposobem jak średnią.

In [ ]:
std19 = d19[['task','duration_type','accuracy']].groupby(['task','duration_type']).std()

Podobnie jest z liczbą obserwacji, którą uzyskamy dzięki metodzie **.count()**.

In [ ]:
n19 = d19[['task','duration_type','accuracy']].groupby(['task','duration_type']).count()

Teraz możemy już wyliczyć błąd standardowy według wzoru.

In [ ]:
err19 = std19 / np.sqrt(n19)
err19 = err19.unstack('task')

In [ ]:
err19

Dane wprowadzamy na wykres jako parametr metody **.plot()**. Należy przy tym pamiętać, że wymiary ramek (w naszym przypadku średniej i błędu standardowego) powinny być ze sobą zgodne, jeśli całość ma nie doprowadzić do błędu.

In [ ]:
acc19.unstack('task').plot(yerr=err19)

Ostatnim krokiem będzie wykorzystanie opcji interaktywnych Jupytera do automatyzacji procesu inspekcji danych poszczególnych osób.  
Będziemy chcieli stworzyć wykres, który pozwoli nam przejrzeć wyniki każdej osoby bez potrzeby pisania nowego kodu czy zmiany tego już istniejącego. Najpierw wybierzmy mniejszą ramkę z danymi o poprawności dla wszystkich osób badanych.

In [ ]:
graph = dane_small[['ID','task','duration_type','accuracy']]

Następnie tworzymy funkcję, która będzie dla nas rysować wykres poprawności dla każdej długości bodźca i warunku zadania. Funkcja ta niech przyjmuje jeden argument - kod osoby badanej.

In [ ]:
def func(k):
    subset = graph[graph.ID == k]
    mean = subset.groupby(['task','duration_type']).mean()
    err = subset.groupby(['task','duration_type']).std() / np.sqrt(subset.groupby(['task','duration_type']).count())
    wykres = mean.stack().unstack('task').plot(yerr = err.stack().unstack('task'))
    
    #trochę ustawień poprawiających estetykę
    wykres.set_title('Poprawnosc wykonania zadania')
    wykres.set_xlim(-0.25,3.25)
    wykres.set_xlabel("Dlugosc bodzca")
    wykres.set_xticklabels(('','12 ms','','50 ms','','82 ms','','153 ms'))
    wykres.set_ylabel("Srednia poprawnosc")

Przetestujmy, czy funkcja spełnia powierzone jej zadanie.

In [ ]:
func('dc1_19')

Następnie musimy zaimportować funkcję <b>interact</b> ze zbioru widget'ów IPythona. Następnie użyjemy tej funkcji podając jej dwa argumenty: funkcję rysującą wykres oraz listę wszystkich kodów osób badanych.

In [ ]:
from ipywidgets import interact

In [ ]:
interact(func,k=list(osoby))

Jak widzimy, otrzymujemy interaktywne okno z rozwijaną listą, z której możemy wybrać osobę badaną a Jupyter sam przeliczy dane i wyświetli nam wynik dla tej konkretnej osoby.

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

## Analiza grupowa

Skoro mamy już pewne pojęcie o manipulacji danymi w ramkach, możemy przejść do analizowania całego zbioru danych.  
Pierwszym krokiem niech będzie przejrzenie wyników poszczególnych osób badanych w celu wyłapania przypadków odstających.

In [ ]:
acc_sort = dane[["ID","accuracy"]].groupby("ID").mean().sort_values('accuracy').reset_index().reset_index()
acc_sort.plot(kind='scatter',x="index", y='accuracy', figsize=(15,5))
plt.tight_layout

To samo możemy przeprowadzić dla oceny na skali PAS.

In [ ]:
pas_sort = dane[["ID","PAS"]].groupby("ID").mean().sort_values('PAS').reset_index().reset_index()
pas_sort.plot(kind='scatter',x="index", y='PAS', figsize=(15,5))
plt.tight_layout

Spróbujmy dołożyć do tego miarę **d'**.

In [ ]:
def mojdprime(row):
    return dprime_mAFC(row,2)

acc_sort['dprime']=0
acc_sort['dprime']=acc_sort['accuracy'].apply(mojdprime)

In [ ]:
acc_sort

In [ ]:
acc_sort.plot(kind='scatter',x="index", y='dprime', figsize=(15,5))
plt.tight_layout

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

### Wykresy grupowe

Przyjrzyjmy się rozkładom odpowiedzi PAS w zależności od zadania i długości prezentacji bodźca (warto to sprawdzić, żeby wiedzieć, czy osoby badane mają wystarczająco dużo prób w każdym warunku aby można było przeprowadzić analizę na danych EEG czy fMRI).

In [ ]:
def tabela_pas(n):
    nowa=dane[dane.accuracy==1].groupby(['ID','PAS','task'])['PAS'].count()
    return nowa[nowa>=n].unstack().unstack().fillna('')
interact(tabela_pas, n=(2,50))

### Wykres Accuracy / Duration

In [ ]:
dane1 = dane_small[['ID','task','duration_type','accuracy']]
print dane1.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).mean().unstack('task')

acc_std=dane1.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).std().unstack('task')
acc_n1=(dane1.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).count().unstack('task')+1).apply(np.sqrt)
acc_se=acc_std/acc_n1
print acc_se

acc_plot=dane1.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).mean().unstack('task').plot(yerr=acc_se, figsize=(15,10))
acc_plot.set_xlim(-0.25,3.25)
acc_plot.set_xticks([0,1,2,3])
acc_plot.set_xticklabels(('12 ms','50 ms','82 ms','153 ms'))
acc_plot.tick_params(axis='both', which='major', labelsize=12)
acc_plot.set_xlabel('Duration', fontsize=16)
acc_plot.set_ylabel('Accuracy', fontsize=16)
#plt.savefig('new_pic/acc_clean_group.png')

### Wykres PAS / Duration

In [ ]:
dane2 = dane_small[['ID','task','duration_type','PAS']]
print dane2.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).mean().unstack('task')

acc_std=dane2.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).std().unstack('task')
acc_n1=(dane2.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).count().unstack('task')+1).apply(np.sqrt)
acc_se=acc_std/acc_n1
print acc_se

acc_plot=dane2.groupby(['ID','task','duration_type']).mean().unstack(['task','duration_type']).mean().unstack('task').plot(yerr=acc_se, figsize=(15,10))
acc_plot.set_xlim(-0.25,3.25)
acc_plot.set_xticks([0,1,2,3])
acc_plot.set_xticklabels(('12 ms','50 ms','82 ms','153 ms'))
acc_plot.tick_params(axis='both', which='major', labelsize=12)
acc_plot.set_xlabel('Duration', fontsize=16)
acc_plot.set_ylabel('Accuracy', fontsize=16)
#plt.savefig('new_pic/acc_clean_group.png')

<a href='#top' style='float: right; font-size: 13px;'>Do początku</a>  

In [ ]:
from scipy.stats import norm
from scipy.integrate import quad
from scipy.special import erf, erfinv
from scipy import Inf
import numpy, scipy
from numpy import exp, log, log10, sign, sqrt
#from numpy.lib.scimath import logn #log with arbitrary base




def dprime_mAFC(Pc, m):
    """
    Compute d' corresponding to a certain proportion of correct
    responses in m-AFC tasks.
    Parameters
    ----------
    Pc : float
        Proportion of correct responses.
    m : int
        Number of alternatives.
    Returns
    -------
    dprime : float
        d' value
    Examples
    --------
    >>> dp = dprime_mAFC(0.7, 3)
    """

    if Pc < 0 or Pc > 1:
        raise ValueError("Pc must be between 0 and 1")
    if isinstance(m, int) == False:
        raise TypeError("m must be an int")
    
    def est_dp(dp):

        def pr(x):
            return (norm.pdf(x-dp) * (norm.cdf(x)**(m-1)))
        
        return (Pc - quad(pr, -Inf, Inf)[0])
    try:
        dprime = scipy.optimize.brentq(est_dp, -10, 10)#scipy.optimize.newton(est_dp, 1)
    except:
        dprime = numpy.nan
    
    return dprime

In [1]:
from IPython.core.display import HTML
import urllib
HTML(urllib.urlopen("https://raw.githubusercontent.com/mkoculak/hello_world/nowa/ipython.css").read())